In [6]:
from datetime import datetime, timedelta
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By



change the language from Chinese to English

In [7]:

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {"translate_whitelists": {"zh-CN": "en"}, "translate": {"enabled": "true"}})  # Translate Chinese to English
driver = webdriver.Chrome(options=options)

# Open the news website
url = "https://www.fromgeek.com/vc/"
driver.get(url)
time.sleep(5)  # Wait for translation to take effect

change the language to English. it says error , then back to change the language to Chinese then it start to change the language aoutomatically

In [9]:
driver.get("https://www.fromgeek.com/en/vc/") 
time.sleep(5)

In [10]:
driver.get("https://www.fromgeek.com/vc/") 
time.sleep(5)

scrape all the necessary information in side each article and each page up to 2 month from now

In [ ]:

# Open CSV file at the start to store scraped data
csv_filename = "updated_scraped_articles.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Article URL', 'Title', 'Date', 'Views', 'Source', 'Content'])  # Write header

# Get the current date and calculate the date 2 months ago
current_date = datetime.now()
two_months_ago = current_date - timedelta(days=60)  # Adjusted to 60 days for 2 months

# Define function to parse date format correctly
def parse_article_date(date_text):
    try:
        return datetime.strptime(date_text, "%Y-%m-%d %H:%M:%S")
    except Exception as e:
        print(f"Error parsing date: {e}")
        return None

# Function to find and click "Next Page" button
def click_next_page():
    try:
        next_buttons = driver.find_elements(By.XPATH, '/html/body/div[2]/div[1]/div/div/div[2]/div[2]/a')
        if next_buttons:
            next_button = next_buttons[-1]
            if next_button.is_displayed() and next_button.is_enabled():
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(3)  # Wait for page load
                return True
        print("No 'Next Page' button found, stopping pagination.")
        return False
    except Exception as e:
        print(f"Error clicking 'Next Page' button: {e}")
        return False

# Start scraping from the first page
page_number = 1

while True:  # Loop through pages until we find an article older than 2 months
    print(f"Scraping page {page_number}...")

    i = 1  # Start scraping articles on the current page
    while True:  # Loop through all articles on the current page
        try:
            # Locate the article link
            article_link = driver.find_element(By.XPATH, f'//*[@id="lists"]/li[{i}]/div[1]/h4/a')
            article_url = article_link.get_attribute('href')

            # Click the article to open
            article_link.click()
            time.sleep(2)

            # Extract the article's date
            date_text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/div/ul/li[2]').text
            article_date = parse_article_date(date_text)

            # Stop scraping if the article is older than 2 months
            if article_date and article_date < two_months_ago:
                print(f"Reached an article older than 2 months ({article_date}), stopping...")
                driver.quit()
                break

            # Extract article details
            title = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/h1').text
            view = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/div/ul/li[1]').text
            source = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/div/ul/li[3]').text
            article_content = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/article/p[1]').text
            # Save data to CSV immediately
            with open(csv_filename, mode='a', newline='', encoding='utf-8') as file: 
                writer = csv.writer(file)
                writer.writerow([article_url, title, article_date.strftime('%Y-%m-%d %H:%M:%S'), view, source, article_content])

            print(f"Scraped: {title} | Date: {article_date}")

            # Go back to the list of articles
            driver.back()
            time.sleep(2.5)

            # Move to the next article
            i += 1

        except Exception as e:
            print(f"Error extracting data for article {i} on page {page_number}: {e}")
            i += 1  # Skip the current article and continue to the next one
            continue  # Ensure the loop does not break on error

    # After scraping all articles on the current page, check if we should continue or stop
    if article_date and article_date < two_months_ago:
        break

    # Click "Next Page" button, if available
    if not click_next_page():
        break

    page_number += 1  # Move to the next page count

print(f"Scraping completed. Data saved to '{csv_filename}'.")


In [1]:
# # Initialize a list to store scraped data
# article_data = []

# # Get the current date and calculate the date 2 months ago
# current_date = datetime.now()
# two_months_ago = current_date - timedelta(days=60)  # Adjusted to 60 days for 2 months

# # Define function to parse date format correctly
# # # Define function to parse date format correctly
# def parse_article_date(date_text):
#     try:
#         # Adjust the date format if needed
#         return datetime.strptime(date_text, "%Y-%m-%d %H:%M:%S")  
#     except Exception as e:
#         print(f"Error parsing date: {e}")
#         return None

# # Function to find and click "Next Page" button
# def click_next_page():
#     try:
#         # Locate all "Next Page" buttons
#         next_buttons = driver.find_elements(By.XPATH, '/html/body/div[2]/div[1]/div/div/div[2]/div[2]/a')

#         # If multiple buttons are found, pick the last one (most likely the correct one)
#         if next_buttons:
#             next_button = next_buttons[-1]

#             # Check if the button is clickable
#             if next_button.is_displayed() and next_button.is_enabled():
#                 driver.execute_script("arguments[0].click();", next_button)  # Use JavaScript to click
#                 time.sleep(3)  # Wait for the next page to load
#                 return True
#             else:
#                 print("Next Page button is not clickable, stopping pagination.")
#                 return False
#         else:
#             print("No 'Next Page' button found, stopping pagination.")
#             return False

#     except Exception as e:
#         print(f"Error clicking 'Next Page' button: {e}")
#         return False

# # Start scraping from the first page
# page_number = 1

# while True:  # Loop through pages until we find an article older than 2 months
#     print(f"Scraping page {page_number}...")

#     i = 1  # Start scraping articles on the current page
#     while True:  # Loop through all articles on the current page
#         try:
#             # Locate the article link
#             article_link = driver.find_element(By.XPATH, f'//*[@id="lists"]/li[{i}]/div[1]/h4/a')
#             article_url = article_link.get_attribute('href')

#             # Click the article to open
#             article_link.click()
#             time.sleep(2)

#             # Extract the article's date
#             date_text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/div/ul/li[2]').text
#             article_date = parse_article_date(date_text)

#             # Stop scraping if the article is older than 2 months
#             if article_date and article_date < two_months_ago:
#                 print(f"Reached an article older than 2 months ({article_date}), stopping...")
#                 driver.quit()
#                 break

#             # Extract article details
#             title = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/h1').text
#             view = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/div/ul/li[1]').text
#             source = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/div/ul/li[3]').text
#             article_content = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div[2]/article/p[2]').text

#             # Append data to list
#             article_data.append([article_url, title, article_date.strftime('%Y-%m-%d %H:%M:%S'), view, source, article_content])

#             # Print extracted data
#             print(f"Scraped: {title} | Date: {article_date}")

#             # Go back to the list of articles
#             driver.back()
#             time.sleep(2.5)

#             # Move to the next article
#             i += 1

#         except Exception as e:
#             print(f"Error extracting data for article {i} on page {page_number}: {e}")
#             break  # Stop on error, or use 'continue' to skip errors

#     # After scraping all articles on the current page, check if we should continue or stop
#     if article_date and article_date < two_months_ago:
#         break

#     # Click "Next Page" button, if available
#     if not click_next_page():
#         break

#     page_number += 1  # Move to the next page count

# # Save the data to a CSV file after scraping
# with open("final_scraped_articles.csv", mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)
#     writer.writerow(['Article URL', 'Title', 'Date', 'Views', 'Source', 'Content']    )
#     writer.writerows(article_data)

# print(f"Scraping completed. Data saved to 'scraped_articles.csv'.")
